### Read and replace webpage
Ongoing notebook to read webpage data, perform LS, generate replacement webpage with LS data

In [7]:
###### import packages, sentence model, and parse html from site url
import requests
import re
from bs4 import BeautifulSoup
import jieba
import torch
import string
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
language_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

url = 'https://www.bbc.com/zhongwen/articles/c4gl97d2rzjo/simp'
site = requests.get(url)
site_soup = BeautifulSoup(site.text, 'html.parser')
# print(site_soup.prettify())

In [6]:
from easse.sari import corpus_sari
from easse.report import get_all_scores
corpus_sari(orig_sents: List[str],
    sys_sents: List[str],
    refs_sents: List[List[str]],)
get_all_scores(orig_sents: List[str],
    sys_sents: List[str],
    refs_sents: List[List[str]],)

In [8]:
##### import CNER model and function
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-news')

2025-01-28 17:12:47,481 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-01-28 17:12:50,144 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-01-28 17:12:50,582 - modelscope - INFO - initiate model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-01-28 17:12:50,584 - modelscope - INFO - initiate model from location C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news.
2025-01-28 17:12:50,593 - modelscope - INFO - initialize model from C:\Users\tempu\.cache\modelscope\hub\damo\nlp_raner_named-entity-recognition_chinese-base-news
2025-01-28 17:12:53,151 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\modelscope\utils\checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will

In [9]:
HSK = pd.read_pickle("../data/Chinese/HSK_full")
# HSK.head()

In [13]:
text = '中国新冠清零政策2022年底结束后，中国政府数次对经济进行刺激，但有节制，效果也不明显。'
punc = ",!?！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.-" # possible punctuation
text.translate(str.maketrans('', '', string.punctuation)) # convert str format
text_re = re.sub(r"[%s]+" %punc, "", text) # remove punctuation marks

tokens = jieba.lcut(text_re, cut_all=False) # tokenize
tokens_l = list(dict.fromkeys(tokens)) #TODO: try: list(set(tokens))

ner_output = ner_pipeline(text)['output'] # add batch to ongoing list
tokens_ner = list(set([d['span'] for d in ner_output if len(d['span'])>1]))
tokens_no_ner = list(set(tokens_l) - set(tokens_ner))

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [46]:
###### get text from the html and tokenize
site_s = site_soup.find_all(string=True) # if paragraph, choose ('p') as func arg
text = ""
punc = ",!?！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.-" # possible punctuation

for s in site_s:
    text += str(s.get_text()) # append new text to the text string

text.translate(str.maketrans('', '', string.punctuation)) # convert str format
text_re = re.sub(r"[%s]+" %punc, "", text) # remove punctuation marks

tokens = jieba.lcut(text_re, cut_all=False) # tokenize
tokens_l = list(dict.fromkeys(tokens)) #TODO: try: list(set(tokens))

In [40]:
def batch(iterable, n=1): # define batching function to run NER pipeline (accepts max 512 characters)
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

ner_output = []
for b in batch(text_re, n=510): # batch text for processing in NER pipeline
    ner_output += ner_pipeline(b)['output'] # add batch to ongoing list
tokens_ner = list(set([d['span'] for d in ner_output if len(d['span'])>1]))
# ner_overlap = list(set(tokens_l) & set(tokens_ner)) # find intersection of NER words and jieba tokens
tokens_no_ner = list(set(tokens_l) - set(tokens_ner))

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\modeling_utils.py:1113: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [15]:
top_choice = torch.tensor(HSK['top_choice'])
top_choice_HSK = torch.tensor(HSK['top_choice_level'])

In [18]:
def simplify(tokens, max_HSK):
    simplified_tokens = dict()
    for token in tokens:
        try:
            char_idx = np.where(HSK["character"]==token)[0][0]
            if HSK["HSK"][char_idx]>max_HSK:
                top_idx = top_choice[char_idx][next(x[0] for x in enumerate(top_choice_HSK[char_idx]) if x[1] <= max_HSK)] # iterate through and find index of first element below max_HSK
                simplified_tokens[token] = HSK["character"].loc[int(top_idx)]
            else:
                pass# simplified_tokens[idx] = 0
        except:
            pass# simplified_tokens[idx] = 1
    return simplified_tokens

In [19]:
replacement_dict = simplify(tokens_no_ner, 4)

In [20]:
replacement_dict

{'政策': '纪律'}

In [21]:
text

'中国新冠清零政策2022年底结束后，中国政府数次对经济进行刺激，但有节制，效果也不明显。'

In [47]:
for element in site_soup.find_all(string=True):  # Get all text nodes
    text = element
    ### Highlight NER in blue:
    for named_entity in tokens_ner:
        text = text.replace(named_entity, f'<span style="color: blue;">{named_entity}</span>')

    ### Highlight replaced words in red:
    for old_word, new_word in replacement_dict.items():
        if old_word in text:
            text = text.replace(old_word, f'<span style="color: red;">{new_word}</span>')

    ### TODO: Highlight words not in HSK list (but not NER) in other color
    newtext = BeautifulSoup(text, 'html.parser')
    element.replace_with(newtext)

In [48]:
with open("output.html", "w", encoding = 'utf-8') as file: 
    # prettify the soup object and convert it into a string 
    file.write(str(site_soup)) 